In [1]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "iaraya"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

 ········


### Loading the instance

In [2]:
from bsg import load_BRinstance
from mclp import generate_candidate_solution
import numpy as np

L,W,H,boxes,id2box = load_BRinstance(filename="BR8.txt",inst=4, nbox=1)
_boxes=boxes.copy()

#Dimension del contenedor
L = int(L/2)
W = int(W/2)
H = int(H/2)


r_param = 1.5 
total_test = 100

### Estimating adjusted vols (optional)

In [ ]:
boxes=_boxes.copy()
#Creacion de solucion candidata
bins1 = generate_candidate_solution(ssh,L,W,H,boxes,id2box, r_param=r_param, bsg_time=1)
boxes=_boxes.copy()
bins2 = generate_candidate_solution(ssh,L,W,H,boxes,id2box, r_param=r_param, bsg_time=1)
boxes=_boxes.copy()
bins3 = generate_candidate_solution(ssh,L,W,H,boxes,id2box, r_param=r_param, bsg_time=1)
boxes=_boxes.copy()
bins4 = generate_candidate_solution(ssh,L,W,H,boxes,id2box, r_param=r_param, bsg_time=1)
boxes=_boxes.copy()
bins5 = generate_candidate_solution(ssh,L,W,H,boxes,id2box, r_param=r_param, bsg_time=1)

from sklearn.linear_model import LinearRegression

# obtengo conjuntos de bins que se pueden colocar en un bin
bins=[*bins1[:-2],*bins2[:-2],*bins3[:-2],*bins4[:-2],*bins5[:-2]] #40 bins

#10x1 + 5x2 +7x3 = 1
#7x3 + 1x7 = 1
#7x3 + 14x1 = 1 
#...


X = []; y=[]
for bin in bins:
    x = np.zeros(len(_boxes))
    i=0
    for box in _boxes:
        if box in bin.boxes:
            x[i]=bin.boxes[box]
        i+=1
    X.append(x)
    y.append(1.0)

reg = LinearRegression(fit_intercept=False).fit(X, y)

i = 0

for box in _boxes:
    vol = box.vol/(L*W*H) 
    if reg.coef_[i] < vol: box.adj_vol=vol
    else: box.adj_vol=reg.coef_[i]
    i+=1
    
print( reg.coef_)

Initial Solution: 10


In [3]:
precomputed_vols = [ 0.11630617,  0.02313203, -0.00057166,  0.02554413,  0.03971392,  0.05794329, \
  0.03677449,  0.04732551,  0.11323912,  0.08207787,  0.02197904,  0.17862389, \
  0.05774166,  0.14886613, -0.03230753,  0.03970321,  0.09253112,  0.0095023, \
  0.10369953,  0.04723096,  0.09009306,  0.06664873,  0.07504355,  0.05946155, \
  0.04751135,  0.08771801,  0.05591064,  0.22794829,  0.07499608,  0.03502724]
i=0
for box in _boxes:
    vol = box.vol/(L*W*H) 
    if precomputed_vols[i] < vol: box.adj_vol=vol
    else: box.adj_vol=precomputed_vols[i]
    i+=1

### Solver

In [4]:
from mclp import get_random_bin

def get_adjusted_vol(boxes):
    vol=0.0
    for box in boxes:
        vol += box.adj_vol*boxes[box]
    return vol
      
def adjusted_swap(solution, n=2,  max_vol_accept=1.0, tolerance=0.1, verbose=True):
    bin_A = get_random_bin(s = solution) 
    bin_B = get_random_bin(s = solution, b = bin_A)
    
    var_ini = (bin_A.adj_vol-1.0)**2 + (bin_B.adj_vol-1.0)**2
    
    nA = int(random.randint(1, n))
    boxes_A = bin_A.pop_random_boxes(nA)
    adjvolA = get_adjusted_vol(boxes_A)
    
    nB = int(random.randint(0, n))
    boxes_B = bin_B.pop_random_boxes(nB)
    adjvolB = get_adjusted_vol(boxes_B)
    
    r = tolerance*random.random()
    accepted = False
    
    if adjvolA > adjvolB and bin_B.adj_vol + adjvolA < max_vol_accept + r:
        accepted = True
    elif adjvolB > adjvolA and  bin_A.adj_vol + adjvolB < max_vol_accept + r:
        accepted = True
    
    if accepted:
        bin_A.insert_boxes(boxes_B)
        bin_B.insert_boxes(boxes_A)
    else: return -10
        
    var_final = (bin_A.adj_vol-1.0)**2 + (bin_B.adj_vol-1.0)**2
        
    var_diff = var_final - var_ini
    
    return var_diff

    

In [5]:
boxes=_boxes.copy()
init_solution=generate_candidate_solution(ssh,L,W,H,boxes,id2box, r_param=r_param, bsg_time=1)


Initial Solution: 10


In [7]:
import copy as cp
for bin in init_solution:
    bin.set_adj_vol(get_adjusted_vol(bin.boxes))
# Bins adjusted volumes
print([get_adjusted_vol(bin.boxes) for bin in init_solution])
best_solution = cp.deepcopy(init_solution)

[0.9714272795003585, 0.8508567899999999, 1.003889542415612, 1.22292749, 0.9927058667175581, 1.2838411837683887, 0.9544956765246557, 1.0361587939825823, 0.9538607579604566, 0.5329067269389196]


In [ ]:
import random
from mclp import verify_solution


for i in range(1000):
    solution = cp.deepcopy(best_solution)
    diff_var = adjusted_swap(solution, n=4,  max_vol_accept=1.02, tolerance=0.1, verbose=True)

    if diff_var>0:
        print(diff_var)
        verified_solution=True
        if not verify_solution(ssh, solution, id2box, bsg_time=15):
            print(i,"verification fails", [b.adj_vol for b in solution if b.verify==False])
            verified_solution=False

        if verified_solution:
            for bin in solution:
                print(bin.id, ":", bin, bin.utilization, bin.adj_vol, bin.verify)
            best_solution = cp.deepcopy(solution)

0.01135548234372819
2 verification fails [1.0387863483829587]
0.0012511937283742705
